In [95]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
from datetime import datetime
from itertools import combinations, permutations
from dateutil.relativedelta import relativedelta
import datetime

In [2]:
!curl https://raw.githubusercontent.com/CorreaWillian/cointegration/testes/ibov.csv -O

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  3 87127    3  2742    0     0   2961      0  0:00:29 --:--:--  0:00:29  2964
100 87127  100 87127    0     0  88871      0 --:--:-- --:--:-- --:--:-- 88905


In [3]:
df = pd.read_excel('PORTFOLIO.xlsx')

In [4]:
df_gpd = df.groupby('ticker').agg({'data_ini': 'min', 'data_fin': 'max', 'setor':'last'})

In [5]:
tickers = df.ticker.unique()

In [6]:
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [7]:
# Download data
df_list = []
errors = []

utc_from = df.data_ini.min() - np.timedelta64(700, 'D')
utc_to = df.data_fin.max()

for i, row in df_gpd.iterrows():
    try: 
        rates = mt5.copy_rates_range(row.name, mt5.TIMEFRAME_M15,utc_from, utc_to)

        # criamos a partir dos dados obtidos DataFrame
        rates_frame = pd.DataFrame(rates)
        # convertemos o tempo em segundos no formato datetime
        rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
        rates_frame['ticker'] = row.name

        df_list.append(rates_frame[['time', 'close', 'ticker']])
    except Exception:
        errors.append(row.name)

In [8]:
errors

['BIDI11', 'GNDI3', 'HGTX3', 'LCAM3']

In [100]:
df_prices = pd.concat(df_list).pivot(columns='ticker',  index='time', values='close')
df_prices.reset_index(inplace=True)

df_prices['time'] = pd.to_datetime(df_prices['time'])

df_prices['horas_minutos'] = df_prices['time'].dt.time

df_prices = df_prices.loc[df_prices['horas_minutos']>=datetime.time(10, 15)]
df_prices = df_prices.loc[df_prices['horas_minutos']<=datetime.time(16, 45)]


In [101]:
df_prices.head()

ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2017-07-31 10:00:00,16.93,11.34,13.00,NaN,26.55,5.75,22.55,14.07,14.16,18.76,...,9.49,9.65,31.88,4.51,23.52,NaN,6.06,26.28,6.77,17.17
2017-07-31 10:15:00,17.07,11.54,12.98,NaN,26.67,5.76,22.60,14.08,14.17,18.70,...,9.47,9.78,31.93,4.50,23.43,NaN,NaN,NaN,6.80,17.17
2017-07-31 10:30:00,17.04,11.57,12.90,NaN,26.46,5.71,22.55,14.04,14.16,18.70,...,9.40,9.83,31.93,4.47,23.43,NaN,NaN,26.08,6.77,17.50
2017-07-31 10:45:00,17.02,11.56,12.85,NaN,26.48,5.69,22.50,14.05,14.11,18.65,...,9.35,9.83,31.93,4.47,23.41,NaN,5.87,NaN,6.78,17.33
2017-07-31 11:00:00,17.01,11.60,12.83,NaN,26.49,5.68,22.50,13.99,14.10,18.64,...,9.32,9.81,31.92,4.47,23.39,NaN,NaN,NaN,6.76,17.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30 15:45:00,13.40,19.19,13.42,14.30,12.36,11.00,33.14,14.34,17.14,25.81,...,12.78,23.30,12.43,8.63,76.34,16.82,1.96,47.08,26.34,13.08
2022-06-30 16:00:00,13.39,19.30,13.41,14.30,12.33,11.02,33.11,14.35,17.14,25.91,...,12.77,23.32,12.40,8.63,76.45,16.78,1.96,47.06,26.35,13.06
2022-06-30 16:15:00,13.41,19.32,13.51,14.30,12.44,11.04,33.25,14.39,17.21,25.92,...,12.79,23.39,12.40,8.71,76.98,16.77,1.97,47.14,26.40,13.10


In [ ]:
df_prices.to_excel('database15min.xlsx')